In [1]:
import sys
import os
import traceback
import time

from numpy import pi
import numpy as np
from qutip import Qobj, basis, ket, mesolve, qeye, tensor, thermal_dm, destroy
from qutip import enr_identity, enr_destroy, enr_thermal_dm
import qutip as qt
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import gridspec

import dimer_phonons as RC
import dimer_optical as EM
from utils import *
import dimer_plotting as vis
import dimer_tests as check
import optical_liouvillian_J as JAKE

matplotlib.style.use('ggplot')

reload(RC)
reload(EM)
reload(vis)
reload(check)

<module 'dimer_tests' from 'dimer_tests.pyc'>

In [ ]:
help(enr_identity)
print np.linalg.eig(np.array([enr_identity([2**4,10,10,10,10], 10).full()]))

Help on function enr_identity in module qutip.operators:

enr_identity(dims, excitations)
    Generate the identity operator for the excitation-number restricted
    state space defined by the `dims` and `exciations` arguments. See the
    docstring for enr_fock for a more detailed description of these arguments.
    
    Parameters
    ----------
    dims : list
        A list of the dimensions of each subsystem of a composite quantum
        system.
    
    excitations : integer
        The maximum number of excitations that are to be included in the
        state space.
    
    state : list of integers
        The state in the number basis representation.
    
    Returns
    -------
    op : Qobj
        A Qobj instance that represent the identity operator in the
        exication-number-restricted state space defined by `dims` and
        `exciations`.



In [49]:
def phonon_dynamics_calculator(w_2, bias, V, T_EM, alpha_EM, alpha_1, alpha_2, 
                               N,exc, optical=True, just_operators=False, new_way=True):
    OO = basis(4,0)
    XO = basis(4,1)
    OX = basis(4,2)
    XX = basis(4,3)
    sigma_m1 = OX*XX.dag() + OO*XO.dag()
    sigma_m2 = XO*XX.dag() + OO*OX.dag()
    sigma_x1 = sigma_m1+sigma_m1.dag()
    sigma_x2 = sigma_m2+sigma_m2.dag()
    w_1 = w_2 + bias
    dipole_1, dipole_2 = 1., 1.
    mu = w_2*dipole_2/(w_1*dipole_1)

    T_1, T_2 = 300., 300. # Phonon bath temperature

    wc = 1*53.08 # Ind.-Boson frame phonon cutoff freq
    w0_2, w0_1 = 500., 501. # underdamped SD parameter omega_0
    w_xx = w_2 + w_1
    N_1, N_2 = N,N # set Hilbert space sizes
    num_cpus = 1
    J = J_minimal
    
    PARAM_names = ['w_1', 'w_2', 'V', 'bias', 'w_xx', 'T_1', 'T_2', 'wc',
                    'w0_1', 'w0_2', 'alpha_1', 'alpha_2', 'N_1', 'N_2', 'exc', 'T_EM', 'alpha_EM','mu', 'num_cpus', 'J', 'dipole_1','dipole_2']
    scope = locals() # Lets eval below use local variables, not global
    PARAMS = dict((name, eval(name, scope)) for name in PARAM_names)
    if just_operators:
        return RC.rate_operators(PARAMS)
    H_dim = w_1*XO*XO.dag() + w_2*OX*OX.dag() + w_xx*XX*XX.dag() + V*(XO*OX.dag() + OX*XO.dag())
    energies, states = check.exciton_states(PARAMS)
    bright_vec = states[1]
    dark_vec = states[0]
    energies_n, states_n = H_dim.eigenstates()

    I_dimer = qeye(4)
    I = enr_identity([N_1,N_2], exc)
    dark = tensor(dark_vec*dark_vec.dag(), I)
    bright = tensor(bright_vec*bright_vec.dag(), I)
    exciton_coherence = tensor(dark_vec*bright_vec.dag(), I)

    '''Defining DM states'''
    site_coherence = tensor(OX*XO.dag(), I)
    OO = tensor(OO*OO.dag(), I)
    XO = tensor(XO*XO.dag(), I)
    OX = tensor(OX*OX.dag(), I)
    XX = tensor(XX*XX.dag(), I)

    #Now we build all of the mapped operators and RC Liouvillian.

    # electromagnetic bath liouvillians

    opts = qt.Options(num_cpus=1, store_states=True)
    ncolors = len(plt.rcParams['axes.prop_cycle'])
    ''' generate the RC liouvillian'''
    L_RC, H_0, A_1, A_2, SIG_1, SIG_2, PARAMS = RC.RC_mapping_OD(PARAMS) # Got the mapped parameters back
    wRC_1, wRC_2, kappa_1, kappa_2 = PARAMS['w0_1'], PARAMS['w0_2'], PARAMS['kappa_1'],PARAMS['kappa_2']
    ''' make the RC observable operators '''
    atemp = enr_destroy([N_1,N_2], exc)
    n_RC_1 = Occupation(wRC_1, T_1)
    n_RC_2 = Occupation(wRC_2, T_2)
    phonon_num_1 = atemp[0].dag()*atemp[0]
    phonon_num_2 = atemp[1].dag()*atemp[1]
    x_1 = (atemp[0].dag()+atemp[0])
    x_2 = (atemp[1].dag()+atemp[1])
    Phonon_1 = tensor(I_dimer, phonon_num_1)
    Phonon_2 = tensor(I_dimer, phonon_num_2)
    disp_1 = tensor(I_dimer, x_1)
    disp_2 = tensor(I_dimer, x_2)

    thermal_RCs = enr_thermal_dm([N_1,N_2], exc, [n_RC_1, n_RC_2])
    ''' initial density operator state'''
    #rho_0 = tensor(basis(4,initial)*basis(4,initial).dag(),thermal_RCs)

    ops = [OO, XO, OX, XX]
    # Expectation values and time increments needed to calculate the dynamics
    expects = ops + [dark, bright, exciton_coherence, site_coherence]
    expects +=[Phonon_1, Phonon_2, disp_1, disp_2]
    
    #timelist = np.linspace(0,end_time,1000*end_time)
    if optical:
        A_EM = SIG_1+PARAMS['mu']*SIG_2
        L_RC += EM.L_non_rwa(H_0, A_EM, PARAMS)
        print "L has shape {}".format(L_RC.shape)
    return L_RC, H_0, expects#, timelist


In [38]:
from numpy.linalg import svd

def compress_L(L, threshold=50.):
    ti = time.time()
    u, s, vh = svd(L.full())
    #print s
    approx_s = [sv if abs(sv)>threshold else 0 for sv in s]
    removed = sum([0. if abs(sv)>threshold else 1. for sv in s])
    print "Removed {} which is {:0.2f}% of the {} singular values.".format(int(removed), 
                                                                           100*(removed/len(approx_s)), 
                                                                            len(approx_s))
    approx_s_mat = np.zeros((u.shape[1], vh.shape[0]), dtype=complex)
    approx_s_mat[:len(s), :len(s)] = np.diag(approx_s)
    approx_L = qt.Qobj(np.dot(u, np.dot(approx_s_mat, vh)), dims = L.dims).tidyup(atol=1e-6)
    print "Compression took {} seconds.".format(time.time()-ti)
    return approx_L

from scipy.sparse.linalg import svds

def compress_L_sp(L, k_fract=0.98):
    ti = time.time()
    k = int(L_RC.shape[0]*k_fract)
    u, s, vh = svds(L.full(), k=k)
    print u.shape, s.shape, vh.shape
    vh = vh[:,:k] # first k columns
    u = u[0:k] # first k rows
    #approx_s = [sv if abs(sv)>1E-8 else 0 for sv in s]
    #kept = sum([1. if abs(sv)>1E-8 else 0. for sv in s])
    #print "Kept {} which is {:0.1f}% of singular values.".format(kept, 100*(kept/len(approx_s)))
    #approx_s_mat = np.zeros((u.shape[1], vh.shape[0]), dtype=complex)
    #approx_s_mat[:len(s), :len(s)] = np.diag(approx_s)
    print "Compression took {} seconds.".format(time.time()-ti)
    a = np.dot(s, vh)
    print a.shape
    return np.dot(u, a)



In [50]:
reload(RC)
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 4,4, new_way=True, optical=True)
#L_RC2, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 5,10, new_way=False)

Building the RC Liouvillian took  2.75445413589 seconds.
Full optical Liouvillian took 2.58380198479 seconds.
L has shape (2704, 2704)


In [48]:
H_0.eigenenergies()

array([    0.        ,   241.22806307,   365.24016344,   664.09152719,
         667.02295221,   667.02295221,   911.732498  ,  1038.34086063,
        1152.39479957,  1289.32521023,  1334.04590442,  1334.04590442,
        1334.04590442,  1545.90533339,  1673.95354386,  1733.26083676,
        1762.80895993,  1854.43736994,  1939.98915646,  2001.06885663,
        2001.06885663,  2001.06885663,  2001.06885663,  2410.15573172,
        2480.99533789,  2574.39596433,  2599.29944006,  2643.96490234,
        2668.09180884,  2668.09180884,  2668.09180884,  2816.41827955,
        2861.71427345,  2933.3066579 ,  3065.97146537,  3119.18348241,
        3203.86713088,  3771.7870563 ,  4120.0705158 ,  4140.32773224,
        4244.89944607,  4266.20480358,  4355.63215583,  4473.97503423,
        4622.81271697,  4759.01662344,  4760.26959383,  4833.03110712,
        4929.62149035,  5310.99287151,  6231.62212176,  6931.95915481])

In [13]:
t = compress_L(L_RC, threshold=1)
t = compress_L(L_RC, threshold=10)
t = compress_L(L_RC, threshold=50)
t = compress_L(L_RC, threshold=100)

[  1.13705456e+05   1.11801940e+05   1.02984101e+05 ...,   7.56500555e-12
   7.56500555e-12   7.56500555e-12]
Removed 9 which is 0.33% of the 2704 singular values.
Compression took 29.1383850574 seconds.
[  1.13705456e+05   1.11801940e+05   1.02984101e+05 ...,   7.56500555e-12
   7.56500555e-12   7.56500555e-12]
Removed 23 which is 0.85% of the 2704 singular values.
Compression took 29.2809779644 seconds.
[  1.13705456e+05   1.11801940e+05   1.02984101e+05 ...,   7.56500555e-12
   7.56500555e-12   7.56500555e-12]
Removed 50 which is 1.85% of the 2704 singular values.
Compression took 29.6318421364 seconds.
[  1.13705456e+05   1.11801940e+05   1.02984101e+05 ...,   7.56500555e-12
   7.56500555e-12   7.56500555e-12]
Removed 77 which is 2.85% of the 2704 singular values.
Compression took 29.1358449459 seconds.


In [25]:
u, s, vt =  svd(L_RC.full())


In [26]:
u.shape

(1024, 1024)

In [43]:
t = compress_L_sp(L_RC, k_fract=0.9)
t = compress_L_sp(L_RC, k_fract=0.95)

KeyboardInterrupt: 

In [190]:
L_test = L_RC
print L_RC
L_test


Quantum object: dims = [[[4, 5, 5], [4, 5, 5]], [[4, 5, 5], [4, 5, 5]]], shape = (5776, 5776), type = super, isherm = False
Qobj data =
[[ 17477.12664965      0.            252.08759734 ...,      0.              0.
       0.        ]
 [     0.          22024.6611616       0.         ...,      0.              0.
       0.        ]
 [  6179.09738481      0.          26572.19567355 ...,      0.              0.
       0.        ]
 ..., 
 [     0.              0.              0.         ...,  11521.12692734
    -463.76747125   -538.08784634]
 [     0.              0.              0.         ...,   4772.53416263
    7886.1511939     377.51311496]
 [     0.              0.              0.         ...,  -4377.69637072
    2402.04085602   7319.35165367]]


TypeError: Can only eliminate states from operators or state vectors

In [184]:
L_c = compress_L_sp(L_RC)

KeyboardInterrupt: 

In [182]:
flat_L = (L_RC.full()).flatten()
print 100*sum([1 if i == 0. else 0 for i in flat_L])/float(len(flat_L)), " percent of matrix is zero."

97.47628  percent of matrix is zero.


In [172]:
print L_RC., L_RC2

Quantum object: dims = [[[4, 5, 5], [4, 5, 5]], [[4, 5, 5], [4, 5, 5]]], shape = (10000, 10000), type = super, isherm = False
Qobj data =
[[  1.74763778e+04   8.71787351e-01   2.51178345e+02 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.92103864e-01   2.20231317e+04   1.64903764e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.17895783e+03   7.26480214e-01   2.65699181e+04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.08026282e+04
    8.64572799e+03  -3.34894611e+02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.12350045e+04
    1.45949529e+04   7.07002318e+02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.41596487e+04
    9.76497327e+03  -1.85938441e+03]] Quantum object: dims = [[[4, 5, 5], [4, 5, 5]], [[4, 5, 5], [4, 5, 5]]], shape = (10000, 10000), type = super, isherm = False
Qobj data =
[[  1.74763778e+04   8.71787351e-01   2.5117

In [205]:
H_0, Z_1, Z_2 = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 1, 1, 5,5, just_operators=True)
print H_0.eigenenergies()
from numpy.linalg import svd
u, s, vh = svd(Z_1.full())
u_t = Qobj(u).eliminate_states([H_0.shape[0]-3])
vh_t= Qobj(vh).eliminate_states([H_0.shape[0]-3])
H_t = qt.Qobj(np.dot(u, np.dot(H_0.full(), vh))).eliminate_states([H_0.shape[0]-3])
print (u_t.dag()*H_t*vh_t.dag()).eigenenergies()
#print qt.Qobj(H_t, dims=H_0.dims).eliminate_states([H_0.shape[0]-1])


[    0.           500.           500.           936.8197497   1000.          1000.
  1000.          1159.72676967  1436.81974972  1437.51872041  1500.          1500.
  1500.          1500.          1659.03228787  1659.72676973  1936.81976202
  1937.51874501  1938.21196904  2000.          2000.          2000.          2000.
  2000.          2096.85840735  2158.34350648  2159.03238118  2159.7268167
  2436.82407711  2437.53171674  2438.22513761  2438.90398581  2500.          2500.
  2500.          2500.          2596.85840739  2596.85840739  2657.67186937
  2658.37889541  2659.06857547  2659.73921422  2936.96887269  2938.05582823
  2939.24357688  2940.1205255   2943.39799642  3096.85840836  3096.85848433
  3096.85848472  3157.19853004  3158.31226145  3159.52229594  3161.95184329
  3165.38360214  3441.74526055  3442.8213935   3443.80409273  3444.87314189
  3596.85860568  3596.85937555  3596.9069347   3596.90695069  3664.09331493
  3664.86934373  3665.6034433   3666.39397749  4096.89108318 

In [186]:
op = compress_L(Z_1) 


[  7.91044961e+03   7.90514549e+03   7.89601407e+03   7.89600955e+03
   7.89589221e+03   7.89075674e+03   7.89075674e+03   7.89054238e+03
   7.46430444e+03   7.46274554e+03   7.45768775e+03   7.45768460e+03
   7.45765713e+03   7.45717112e+03   7.45717112e+03   7.45713463e+03
   7.44826372e+03   7.44467947e+03   7.43135841e+03   7.42677506e+03
   6.97645031e+03   6.97451994e+03   6.96482138e+03   6.96436264e+03
   6.95749187e+03   6.95503788e+03   6.93870274e+03   6.93478718e+03
   6.83607443e+03   6.83551623e+03   6.83457678e+03   6.83457375e+03
   6.83456470e+03   6.83398790e+03   6.83398790e+03   6.83396529e+03
   6.45420264e+03   6.45227905e+03   6.43753009e+03   6.43712921e+03
   6.43137519e+03   6.42966260e+03   6.41181883e+03   6.40856348e+03
   6.32896278e+03   6.32887558e+03   6.32853387e+03   6.32847012e+03
   6.32846431e+03   6.32840816e+03   6.32840816e+03   6.32833760e+03
   6.32666890e+03   6.32632342e+03   6.32519861e+03   6.32479758e+03
   5.88777991e+03   5.88611006e+03

In [187]:
print ((H_0.shape[0]-24)**2)/float(H_0.shape[0])**2, Xi_1.shape

0.8212890625 (212, 212)


In [151]:
Chi_1, Xi_1, Chi_2, Xi_2 = compress_L(Chi_1), compress_L(Xi_1),compress_L(Chi_2),compress_L(Xi_2)

[  6.38093352e+03   6.38014473e+03   6.37875956e+03   6.37873959e+03
   6.37809086e+03   6.37806982e+03   6.37804746e+03   6.37769192e+03
   6.37767922e+03   6.37764168e+03   6.37611875e+03   6.37611875e+03
   6.37611875e+03   6.37611875e+03   6.37606431e+03   6.37605089e+03
   5.86211913e+03   5.86119950e+03   5.86000893e+03   5.85999168e+03
   5.85769771e+03   5.85736476e+03   5.85588594e+03   5.85588594e+03
   5.30674735e+03   5.30576669e+03   5.30475453e+03   5.30473852e+03
   5.30068707e+03   5.30041876e+03   5.29904624e+03   5.29904624e+03
   4.70611561e+03   4.70512646e+03   4.70428023e+03   4.70426626e+03
   4.69834631e+03   4.69814043e+03   4.69688762e+03   4.69688762e+03
   4.53215596e+03   4.53205937e+03   4.53085430e+03   4.53084931e+03
   4.53028811e+03   4.53027378e+03   4.53025474e+03   4.53021520e+03
   4.53020401e+03   4.53018153e+03   4.52899334e+03   4.52899334e+03
   4.52899334e+03   4.52899334e+03   4.52895199e+03   4.52894892e+03
   4.04742922e+03   4.04650104e+03

In [168]:
%timeit (Chi_1+Xi_1).dag()

%timeit Chi_1-Xi_1

1000 loops, best of 3: 203 µs per loop
1000 loops, best of 3: 235 µs per loop


#### L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 4,5)
L_approx = compress_L(L_RC)
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 4,4)
L_approx = compress_L(L_RC)
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 3,3)
L_approx = compress_L(L_RC)
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 3,4)
L_approx = compress_L(L_RC)
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 4,3)
L_approx = compress_L(L_RC)



"""
L_approx = compress_L(L_RC)
del L_RC, H_0, L_approx, expects
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 100, 50, 6, 6)
L_approx = compress_L(L_RC)
del L_RC, H_0, L_approx, expects
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 100, 50, 7, 7)
L_approx = compress_L(L_RC)
del L_RC, H_0, L_approx, expects
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 100, 50, 8, 8)
L_approx = compress_L(L_RC)
del L_RC, H_0, L_approx, expects
"""



In [68]:

L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 3,3)
ti = time.time()
u, s, vh = svd(H_0.full())
print s, time.time()- ti
ti = time.time()
print sorted(H_0.eigenenergies()) ,time.time()- ti

L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 3,6)
ti = time.time()
u, s, vh = svd(H_0.full())
print s, time.time()- ti
ti = time.time()
print sorted(H_0.eigenenergies()) ,time.time()- ti


Building the RC Liouvillian took  1.07638502121 seconds.
Full optical Liouvillian took 1.17074418068 seconds.
[  5.74843612e+03   4.75790404e+03   3.95729385e+03   3.91312464e+03
   3.83821719e+03   3.61003399e+03   3.55661930e+03   3.46716145e+03
   3.26386816e+03   3.24333561e+03   3.16854620e+03   2.26310253e+03
   2.15916182e+03   2.10725995e+03   2.00106886e+03   2.00106886e+03
   1.96699956e+03   1.93419969e+03   1.87336101e+03   1.60252691e+03
   1.47083067e+03   1.33404590e+03   1.33404590e+03   1.33404590e+03
   1.12186010e+03   9.94358252e+02   8.24027307e+02   6.67022952e+02
   6.67022952e+02   4.55486946e+02   3.17248713e+02   4.73981874e-13] 0.00097918510437
[0.0, 317.24871288892638, 455.48694575653445, 667.02295221018483, 667.02295221018483, 824.0273067857197, 994.35825209428822, 1121.860099206473, 1334.0459044203697, 1334.0459044203699, 1334.0459044203699, 1470.8306675384758, 1602.5269118068893, 1873.3610063790529, 1934.1996866295942, 1966.9995616748424, 2001.06885663055

In [63]:

"""L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 5,2)
L_approx = compress_L(L_RC)"""

Building the RC Liouvillian took  1.02686190605 seconds.
Full optical Liouvillian took 0.82214307785 seconds.
[  6.46263528e+04   6.24806987e+04   5.95546645e+04 ...,   4.90909953e-12
   4.90909953e-12   4.64592362e-12]
Removed 4 which is 0.39% of the 1024 singular values.
Compression took 1.8036608696 seconds.
Building the RC Liouvillian took  1.2979259491 seconds.
Full optical Liouvillian took 1.08403611183 seconds.
[  5.98761795e+04   5.98744854e+04   5.98580592e+04 ...,   1.15074034e-12
   1.09385706e-12   7.92404165e-13]
Removed 36 which is 2.78% of the 1296 singular values.
Compression took 3.26391196251 seconds.
Building the RC Liouvillian took  1.56289887428 seconds.
Full optical Liouvillian took 1.39505887032 seconds.
[  8.13873229e+04   7.92623066e+04   7.40049764e+04 ...,   6.11578255e-12
   6.11578255e-12   6.11578255e-12]
Removed 4 which is 0.25% of the 1600 singular values.
Compression took 6.32245087624 seconds.
Building the RC Liouvillian took  1.52168798447 seconds.
Fu

'L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 500, 500, 5,2)\nL_approx = compress_L(L_RC)'

In [33]:
L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 100, 50, 3, 3)
#L_approx = compress_L_sp(L_RC)
#del L_RC, H_0, L_approx, expects
#L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 100, 50, 6, 6)
#L_approx = compress_L_sp(L_RC)
#del L_RC, H_0, L_approx, expects
#L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 100, 50, 7, 7)
#L_approx = compress_L_sp(L_RC)
#del L_RC, H_0, L_approx, expects
#L_RC, H_0, expects = phonon_dynamics_calculator(1000, 100, 100, 300, 5, 100, 50, 8, 8)
#L_approx = compress_L_sp(L_RC)
#del L_RC, H_0, L_approx, expects

Building the RC Liouvillian took  14.2370569706 seconds.


In [40]:
print L_RC.shape, L_RC.dims

(10816, 10816) [[[4, 6, 6], [4, 6, 6]], [[4, 6, 6], [4, 6, 6]]]


array([[    0.00000000+0.j,     0.00000000+0.j,     0.00000000+0.j, ...,
            0.00000000+0.j,     0.00000000+0.j,     0.00000000+0.j],
       [    0.00000000+0.j,   667.02295221+0.j,     0.00000000+0.j, ...,
            0.00000000+0.j,     0.00000000+0.j,     0.00000000+0.j],
       [    0.00000000+0.j,     0.00000000+0.j,  1334.04590442+0.j, ...,
            0.00000000+0.j,     0.00000000+0.j,     0.00000000+0.j],
       ..., 
       [    0.00000000+0.j,     0.00000000+0.j,     0.00000000+0.j, ...,
         4101.06885663+0.j,     0.00000000+0.j,     0.00000000+0.j],
       [    0.00000000+0.j,     0.00000000+0.j,     0.00000000+0.j, ...,
            0.00000000+0.j,  3434.04590442+0.j,   228.88394474+0.j],
       [    0.00000000+0.j,     0.00000000+0.j,     0.00000000+0.j, ...,
            0.00000000+0.j,   228.88394474+0.j,  4101.06885663+0.j]])

In [32]:
from qutip import sigmax, 